In [11]:
# Import the function from gpu.py
import sys
sys.path.append('./translate')  # Add the module path to the system path
from gpu import translate_with_gpu

In [13]:
from translate import *

In [14]:
# Test input
test_input = "안녕하세요, 우주는 얼마나 넓나요?"

# Call the translate_text function
out_text = translate_text(test_input)
out_text

'Hello, how vast is the universe?'

In [ ]:
# translate/common.py
from environment import detect_environment
from translate.gpu import translate_with_gpu
from translate.furiosa import translate_with_furiosa

def translate_text(input_text, source_language="ko", target_language="en"):
    """
    Translates text based on the detected environment.

    Args:
        input_text (str): Text to be translated.
        source_language (str): Source language (default: "ko").
        target_language (str): Target language (default: "en").

    Returns:
        str: Translated text.
    """
    environment = detect_environment()

    if environment == "gpu":
        return translate_with_gpu(input_text, source_language, target_language)
    elif environment == "furiosa":
        return translate_with_furiosa(input_text, source_language, target_language)
    else:
        raise EnvironmentError(
            "No suitable environment detected. Please ensure GPU or Furiosa RNGD is available."
        )

In [23]:
r = calculate_bleu_meteor(reference_text, candidate_text)
print(r)

{'BLEU': 0.0103, 'METEOR': 0.1594}


In [2]:
# quality_metrics.py에서 필요한 함수 임포트
from metrics.quality_metrics import calculate_bleu_meteor_score, calculate_bert_score

# 예제 텍스트 설정
reference_text = "Increasingly supermarkets are getting a more varied section of ready-made food. Some even provide a microwave oven or other means to heat food."
candidate_text = "At the supermarket, the variety of ready-to-eat foods has increased. The store has also begun to offer services such as using electronic money."

# BLEU 점수 계산
bleu_score, meteor_score, num_tokens = calculate_bleu_meteor_score(reference_text, candidate_text)
print(f"BLEU Score: {bleu_score}")
print(f"METEOR Score: {meteor_score}")

# BERTScore 계산
try:
    bert_score = calculate_bert_score(reference_text, candidate_text, tgt_lang="English", device="cuda")
    print(f"BERTScore (F1): {bert_score}")
except Exception as e:
    print(f"Error calculating BERTScore: {e}")

BLEU Score: 0.0103
METEOR Score: 0.1594


/home/dudaji/miniconda3/envs/langserve/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BERTScore (F1): 0.7193


In [25]:
from translate.one_trans import initialize_translation_environment, translate_text
import time
from metrics.quality_metrics import calculate_bleu_meteor_score, calculate_bert_score


# Initialize the translation environment
try:
    initialize_translation_environment()
except EnvironmentError as e:
    print(f"Failed to initialize translation environment: {e}")
    exit(1)

# Perform translation
test_input = "안녕하세요, 번역 테스트입니다."
ref_text = "Hi, this is a test for translation"

In [26]:
start_time = time.time()
result = translate_text(test_input)
time_translate = time.time() - start_time

print("Translated Text:", result)


Translated Text: Hello, this is a translation test.


In [32]:
from metrics.quality_metrics import tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# 텍스트를 토큰화
reference_tokens = tokenize(ref_text, "English")
candidate_tokens = tokenize(result, "English")
smoothing_function = SmoothingFunction().method1


In [38]:
candidate_tokens

['Hello', ',', 'this', 'is', 'a', 'translation', 'test', '.']

In [49]:
from nltk.translate.meteor_score import meteor_score
reference_str = " ".join(reference_tokens)  # 리스트 -> 문자열 변환
candidate_str = " ".join(candidate_tokens)  # 리스트 -> 문자열 변환
# meteor = meteor_score([reference_str], candidate_str)
meteor = meteor_score([reference_tokens], candidate_tokens)
meteor


0.840561224489796

In [ ]:
num_tokens = len(candidate_tokens)

# 지표 계산
bleu = sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoothing_function)
meteor = meteor_score([reference_tokens], candidate_tokens)

In [ ]:
from translate.one_trans import initialize_translation_environment, translate_text
initialize_translation_environment()

In [29]:
bleu_score, meteor_score = calculate_bleu_meteor_score(ref_text, result)
bleu_score, meteor_score

(0.3826, 0.8406)

In [1]:
from translate.one_trans import initialize_translation_environment, translate_text
import time
from metrics.quality_metrics import calculate_bleu_meteor_score, calculate_bert_score
from metrics.evaluate_translation import evaluate_translation


# Initialize the translation environment
try:
    initialize_translation_environment()
except EnvironmentError as e:
    print(f"Failed to initialize translation environment: {e}")
    exit(1)

# Perform translation
test_input = "안녕하세요, 번역 테스트입니다."
ref_text = "Hi, this is a test for translation"
try:
    start_time = time.time()
    result = translate_text(test_input)
    time_translate = time.time() - start_time
    
    bleu_val, meteor_val, num_tokens = calculate_bleu_meteor_score(ref_text, result)
    
    print("Translated Text:", result)
    print(f"BLEU Score: {bleu_val}")
    print(f"METEOR Score: {meteor_val}")
    print("TPS:", num_tokens/time_translate)
except RuntimeError as e:
    print(f"Translation failed: {e}")

2024-12-30 19:01:55,297 - INFO - Translation environment detected: GPU. Using GPU for translations.
2024-12-30 19:01:55,618 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


Translated Text: Hello, this is a translation test.
BLEU Score: 0.3826
METEOR Score: 0.8406
TPS: 24.951911264727492


In [1]:
from time import time
from translate.one_trans import initialize_translation_environment, translate_text
from metrics.load_config import load_active_metrics
from metrics.evaluation_transition import evaluate_translation

# Example usage
config_path = "/home/dudaji/Jun/llm-rag-chatbot/main/config.yaml"
active_metrics = load_active_metrics(config_path)
print("Active Metrics:", active_metrics)

# Test data
test_input = "안녕하세요, 번역 테스트입니다."
ref_text = "Hi, this is a test for translation"

# Initialize the translation environment
try:
    initialize_translation_environment()
except EnvironmentError as e:
    print(f"Failed to initialize translation environment: {e}")
    exit(1)
    
    
# Translation
try:
    start_time = time()
    translated_text = translate_text(test_input)
    elapsed_time = time() - start_time
except RuntimeError as e:
    print(f"Translation failed: {e}")
    translated_text = None
    elapsed_time = 0.0

# Evaluation
if translated_text:
    metrics = evaluate_translation(translated_text, ref_text, elapsed_time, active_metrics)
    print("Metrics:", metrics)

Active Metrics: ['BLEU', 'METEOR', 'BERTScore', 'tps']


2024-12-30 19:39:57,114 - INFO - Translation environment detected: GPU. Using GPU for translations.
2024-12-30 19:39:57,430 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
/home/dudaji/miniconda3/envs/langserve/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Metrics: {'BLEU': 0.3826, 'METEOR': 0.8406, 'BERTScore': 0.8641}


In [56]:
from translate.one_trans import initialize_translation_environment, translate_text
import time
from metrics.quality_metrics import calculate_bleu_meteor_score, calculate_bert_score

reference = "Increasingly supermarkets are getting a more varied section of ready-made food."
candidate = "At the supermarket, the variety of ready-to-eat foods has increased."

# 함수 호출
try:
    bleu_val, meteor_val = calculate_bleu_meteor_score(reference, candidate, tgt_lang="English")
    print("BLEU Score:", bleu_val)
    print("METEOR Score:", meteor_val)
    print("Number of Tokens:", num_tokens)
except ValueError as e:
    print(f"Error occurred: {e}")

BLEU Score: 0.0203
METEOR Score: 0.1667
Number of Tokens: 8
